In [ ]:
import os
import torch
import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.tv_tensors as tv
from torchinfo import summary
import torch.nn as nn
from PIL import Image

# Load a pre-trained ResNet-18 model
model = models.resnet18(weights='ResNet18_Weights.IMAGENET1K_V1')

# Freeze all the parameters
for param in model.parameters():
    param.requires_grad = False

# Modify the first convolutional layer to accept 1-channel input
model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)

# Unfreeze the parameters of the first convolutional layer
for param in model.conv1.parameters():
    param.requires_grad = True

# Get the number of features in the last layer
num_features = model.fc.in_features

# Modify the last fully connected layer to have the same number of output classes as your dataset
model.fc = nn.Linear(num_features, 15)

# Ensure the parameters of the fully connected layer are trainable
for param in model.fc.parameters():
    param.requires_grad = True

summary(model)

In [ ]:
ROOT = "/kaggle/input/amia-public-challenge-2024"

test_img_path = ROOT + "/test/test"
train_img_path = ROOT + "/train/train"

test_annot_path = ROOT + "/test.csv"
train_annot_path = ROOT + "/train.csv"

In [ ]:
import pandas as pd

# Load the train CSV file
train_df = pd.read_csv(train_annot_path)
test_df = pd.read_csv(test_annot_path)
print(train_df.head())
print(train_df.columns)

In [11]:
# Load the Json file
import json

dict_path = 'pre-pro/image_dict.json'

with open(dict_path) as f:
    train_dict = json.load(f)

# print first enry of the dictionary
print(list(train_dict.items())[0])

('bM8C97htulC9fHKIDurJHquCXr1KZuug', {'classes': {'14.0': {'R5': [[0.0, 0.0, 0.0005015045135406219, 0.00041152263374485596]], 'R3': [[0.0, 0.0, 0.0005015045135406219, 0.00041152263374485596]], 'R17': [[0.0, 0.0, 0.0005015045135406219, 0.00041152263374485596]]}}, 'og_dims': [2430, 1994]})


In [ ]:
# Group by image_id to ensure each image stays in one set
unique_image_ids = train_df['image_id'].unique()
print(f"Unique Train IDs: {len(unique_image_ids)}")

from torch.utils.data import random_split
# Split into train and validation sets
train_split = 0.7
train_size = int(train_split * len(unique_image_ids))
val_size = len(unique_image_ids) - train_size

# Ensure reproducibility
torch.manual_seed(42)

# Get the indices of the train and validation sets
train_indices, val_indices = random_split(unique_image_ids.tolist(), [train_size, val_size])
print(f"Train indices: {len(train_indices)}")
print(f"Val indices: {len(val_indices)}")

# Get the corresponding image IDs
train_ids = [unique_image_ids[i] for i in train_indices.indices]
val_ids = [unique_image_ids[i] for i in val_indices.indices]

# Create train and validation dataframes
train_data_df = train_df[train_df['image_id'].isin(train_ids)]
val_data_df = train_df[train_df['image_id'].isin(val_ids)]

# Check the number of unique images in each set
train_unique_images = train_data_df['image_id'].nunique()
val_unique_images = val_data_df['image_id'].nunique()

print(f"Training samples: {train_data_df.shape[0]}, Training unique images: {train_unique_images}")
print(f"Validation samples: {val_data_df.shape[0]}, Validation unique images: {val_unique_images}")

In [ ]:
from torch.utils.data import Dataset
import pandas as pd
import numpy as np
from tqdm import tqdm

class XRayImageDataset(Dataset):
    
    def __init__(self, dict , img_dir, size, mean=None, std=None,transform_norm=None, target_transform=None):
        self.dict = dict
        self.img_dir = img_dir
        self.keys = list(dict.keys())
        self.size = size
        self.mean = mean
        self.std = std
        self.transform_norm = transforms.Compose([
            transforms.Resize((224,224),antialias=True),
            transforms.Normalize(self.mean, self.std)
        ])
        self.target_transform = target_transform    

    def __len__(self):
        return len(self.keys)

    def __getitem__(self, idx):
        img_id = self.keys[idx]
        img_path = os.path.join(self.img_dir, img_id) + '.png'
        image = read_image(img_path).float() # PyTorch function, no need to change
        image = self.transform_norm(image)
        # https://pytorch.org/tutorials/intermediate/torchvision_tutorial.html
        
        
        box_list = []
        label_list = []
        area_list = []
        iscrowd_list = []
        # get all boxes of all radiologists
        # structure of the dict
        # img_id: {
        #     "classes": [
        #         class_id: [
        #            rad_id: [[bbox],[bbox]]]
        
        for class_id in self.dict[img_id]["classes"].items():
            for rad in class_id.items():
                for box in rad.items():
                    box = box * self.size
                    box_list.append(box)
                    label_list.append(class_id)
                    area_list.append((box[2]-box[0])*(box[3]-box[1])) # x_max-x_min * y_max-y_min
                    iscrowd_list.append(0)
        
        # target dict
        target = {
            "boxes": tv.BoundingBoxes(box_list),
            "labels": label_list,
            "image_id": torch.tensor([idx]),
            "area": torch.tensor(area_list),
            "iscrowd": torch.tensor(iscrowd_list)
        }

        return image, target, img_id

In [ ]:
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torchvision import datasets, transforms
from torchvision.io import read_image

train_mean = 0.04664242120787185
train_std = 0.10213025072799406
# TODO: improvement for separate train / val computations?

train_batch_size = 64

train_dataset = XRayImageDataset(train_data_df, train_img_path, train_mean, train_std)
print(f'Training images: mean {train_dataset.mean}, std {train_dataset.std}')
train_dataloader = DataLoader(train_dataset, batch_size=train_batch_size, shuffle=True)

val_dataset = XRayImageDataset(val_data_df, train_img_path, train_mean, train_std)
val_dataloader = DataLoader(val_dataset, batch_size=train_batch_size*2, shuffle=True)
print(f'Using same metrics for validation at {train_split}/{1-train_split} ratio')

In [ ]:
# TODO: use mean and std from training set?
test_mean = 0.07274098403775907
test_std = 0.16118353533641264

test_batch_size = 32

test_dataset = XRayImageDataset(test_df, test_img_path, test_mean, test_std)
print(f'Testing images: mean {test_dataset.mean}, std {test_dataset.std}')
test_dataloader = DataLoader(test_dataset, batch_size=test_batch_size, shuffle=True)

In [ ]:
import torch.optim as optim
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Number of epochs
num_epochs = 10

# Device configuration (use GPU if available)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Training and evaluation loop
for epoch in range(num_epochs):
    # Training phase
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in tqdm(train_dataloader, desc=f'Train Ep. {epoch+1}'):
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        # Statistics
        running_loss += loss.item() * inputs.size(0)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / total
    epoch_acc = correct / total

    print(f'Train Loss: {epoch_loss:.4f}, Train Acc: {epoch_acc:.4f}')

    # Validation phase
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in tqdm(val_dataloader, desc=f'Validating Ep. {epoch+1}/{num_epochs}'):
            inputs, labels = inputs.to(device), labels.to(device)

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Statistics
            val_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_loss /= total
    val_acc = correct / total

    print(f'Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_acc:.4f}')

print('Training complete')